In [9]:
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [10]:
# Load dataset
df = pd.read_csv('C:/Users/Kornelius/Desktop/Data 2/nips-papers/papers.csv', header = 0, sep = ';', error_bad_lines=False)
# Drop the columns not needed
df = df.drop(columns=['id', 'event_type', 'pdf_name'], axis=1)

In [11]:
# Drop a row by condition
df = df[df.abstract != 'Abstract Missing']
# Print out the first rows of papers

In [12]:
data = df.abstract.values.tolist()

In [13]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(data):
    stop_free = " ".join([i for i in data.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

data = [clean(data).split() for data in data]  

In [14]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data)) # list in front
data_words

[['how',
  'information',
  'decoded',
  'brain',
  'one',
  'difficult',
  'important',
  'question',
  'neuroscience',
  'whether',
  'neural',
  'correlation',
  'important',
  'decoding',
  'neural',
  'activity',
  'special',
  'interest',
  'developed',
  'general',
  'framework',
  'investigating',
  'far',
  'decoding',
  'process',
  'brain',
  'simplified',
  'first',
  'hierarchically',
  'construct',
  'simplified',
  'probabilistic',
  'model',
  'neural',
  'response',
  'ignore',
  'kthorder',
  'correlation',
  'using',
  'maximum',
  'entropy',
  'principle',
  'then',
  'compute',
  'much',
  'information',
  'lost',
  'information',
  'decoded',
  'using',
  'simplified',
  'model',
  'ie',
  'mismatched',
  'decoder',
  'introduce',
  'information',
  'theoretically',
  'correct',
  'quantity',
  'evaluating',
  'information',
  'obtained',
  'mismatched',
  'decoder',
  'applied',
  'proposed',
  'framework',
  'spike',
  'data',
  'vertebrate',
  'retina',
  'used

In [16]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(16129 unique tokens: ['activity', 'also', 'analysis', 'applied', 'assume']...)


In [17]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
lda_model5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [20]:
# Print the Keyword in the 1 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"algorithm" + 0.018*"model" + 0.014*"learning" + 0.013*"problem" + '
  '0.013*"method" + 0.012*"data" + 0.010*"show" + 0.008*"result" + '
  '0.007*"approach" + 0.007*"function"')]


In [21]:
# Print the Keyword in the 5 topics
pprint(lda_model5.print_topics())
doc_lda = lda_model5[corpus]

# Print the Keyword in the 10 topics
pprint(lda_model10.print_topics())
doc_lda = lda_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lda_model20.print_topics())
doc_lda = lda_model20[corpus]

# Print the Keyword in the 50 topics
pprint(lda_model50.print_topics())
doc_lda = lda_model50[corpus]

# Print the Keyword in the 100 topics
pprint(lda_model100.print_topics())
doc_lda = lda_model100[corpus]

[(0,
  '0.037*"learning" + 0.037*"data" + 0.036*"method" + 0.021*"approach" + '
  '0.013*"task" + 0.012*"propose" + 0.012*"framework" + 0.012*"label" + '
  '0.012*"proposed" + 0.011*"feature"'),
 (1,
  '0.040*"algorithm" + 0.025*"problem" + 0.015*"function" + 0.014*"show" + '
  '0.013*"result" + 0.012*"bound" + 0.009*"study" + 0.009*"matrix" + '
  '0.008*"setting" + 0.008*"learning"'),
 (2,
  '0.045*"graph" + 0.038*"model" + 0.027*"signal" + 0.021*"clustering" + '
  '0.021*"system" + 0.017*"neuron" + 0.016*"information" + 0.014*"dynamic" + '
  '0.014*"node" + 0.014*"cluster"'),
 (3,
  '0.058*"network" + 0.051*"image" + 0.048*"model" + 0.023*"deep" + '
  '0.022*"neural" + 0.021*"object" + 0.020*"word" + 0.014*"training" + '
  '0.012*"learn" + 0.011*"visual"'),
 (4,
  '0.087*"model" + 0.042*"distribution" + 0.027*"inference" + 0.020*"variable" '
  '+ 0.018*"bayesian" + 0.018*"process" + 0.017*"estimator" + '
  '0.017*"estimation" + 0.014*"gaussian" + 0.013*"latent"')]
[(0,
  '0.051*"meth

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity5: ', lda_model5.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity10: ', lda_model10.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity20: ', lda_model20.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity50: ', lda_model50.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity100: ', lda_model100.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100: ', coherence_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence ScoreUMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100UMass: ', coherence_lda)


Perplexity:  -6.505842040207014

Perplexity5:  -6.419870567349887

Perplexity10:  -6.414332017641584

Perplexity20:  -6.4161997530349595

Perplexity50:  -6.390714832314033

Perplexity100:  -9.713165114091508

Coherence Score:  0.2357683522987768

Coherence Score5:  0.4109556072553883

Coherence Score10:  0.43944221339262707

Coherence Score20:  0.36274370735310413

Coherence Score50:  0.25770205981477495

Coherence Score100:  0.2201411730989503

Coherence ScoreUMass:  -1.4635242209070833

Coherence Score5UMass:  -2.0302924691264606

Coherence Score10UMass:  -2.4484281921230178

Coherence Score20UMass:  -3.5110651458415445

Coherence Score50UMass:  -4.530041195668397

Coherence Score100UMass:  -4.504189216274774
